In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [7]:
df = pd.read_csv('./pokemon_alopez247.csv')

# Formatting Data

In [8]:
df.columns

Index(['Number', 'Name', 'Type_1', 'Type_2', 'Total', 'HP', 'Attack',
       'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Generation', 'isLegendary',
       'Color', 'hasGender', 'Pr_Male', 'Egg_Group_1', 'Egg_Group_2',
       'hasMegaEvolution', 'Height_m', 'Weight_kg', 'Catch_Rate',
       'Body_Style'],
      dtype='object')

In [12]:
# select categories that will be relevant
df = df[['isLegendary','Generation', 'Type_1', 'Type_2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed','Color','Egg_Group_1','Height_m','Weight_kg','Body_Style']]

In [13]:
# convert this column from boolean "False" and "True" statements to the equivalent "0" and "1" integers
df['isLegendary'] = df['isLegendary'].astype(int)

In [16]:
print(df['isLegendary'])

0      0
1      0
2      0
3      0
4      0
      ..
716    1
717    1
718    1
719    1
720    1
Name: isLegendary, Length: 721, dtype: int64


df.head()

In [21]:
# A few other categories need to create dummy variables

'''
This function first uses pd.get_dummies to create a dummy DataFrame of that category. 
As it's a seperate DataFrame, we'll need to concatenate it to our original DataFrame. 
And since we now have the variables represented properly as separate columns, we drop the original column. 
Having this in a function is nice because we can quickly do this for many categories:
'''

def dummy_creation(df, dummy_categories):
    for i in dummy_categories:
        df_dummy = pd.get_dummies(df[i])
        df = pd.concat([df,df_dummy],axis=1)#concatenate the new dummied variables to originial dataframe
        df = df.drop(i, axis=1)#drop the original column
    return(df)

df = dummy_creation(df, ['Egg_Group_1', 'Body_Style', 'Color','Type_1', 'Type_2'])


# Split and Normalize data

In [24]:
'''
In general, the data is randomly split with about 70% being used for training and 30% used for testing. 
For easier visualization, we'll be splitting the data by Pokémon generation. 
The first generation of Pokémon (from Pokémon Red, Blue, and Yellow) will be our testing data while the rest will be our training data:
'''

def train_test_splitter(DataFrame, column):
    df_train = DataFrame.loc[df[column] != 1]
    df_test = DataFrame.loc[df[column] == 1]

    df_train = df_train.drop(column, axis=1)
    df_test = df_test.drop(column, axis=1)

    return(df_train, df_test)

df_train, df_test = train_test_splitter(df, 'Generation')

'''
This function takes any Pokémon whose "Generation" label is equal to 1 and putting it into the test dataset, 
and putting everyone else in the training dataset. 
It then drops the Generation category from the dataset.
'''


'\nThis function takes any Pokémon whose "Generation" label is equal to 1 and putting it into the test dataset, \nand putting everyone else in the training dataset. \nIt then drops the Generation category from the dataset.\n'

Now that we have our two sets of data, we'll need to separate the labels (the 'islegendary' category) from the rest of the data. 
Remember, this is the answer key to the test the algorithms are trying to solve, and it does no good to have them learn with the answer-key in (metaphorical) hand:

In [27]:


def label_delineator(df_train, df_test, label):
    
    train_data = df_train.drop(label, axis=1).values
    train_labels = df_train[label].values
    test_data = df_test.drop(label,axis=1).values
    test_labels = df_test[label].values
    return(train_data, train_labels, test_data, test_labels)

# This function extracts the data from the DataFrame and puts it into arrays that TensorFlow can understand with.values. 


In [28]:
#We then have the four groups of data:
train_data, train_labels, test_data, test_labels = label_delineator(df_train, df_test, 'isLegendary')

In [33]:
# normalize the data so everything is on the same scale

def data_normalizer(train_data, test_data):
    train_data = preprocessing.MinMaxScaler().fit_transform(train_data)
    test_data = preprocessing.MinMaxScaler().fit_transform(test_data)
    return(train_data, test_data)

train_data, test_data = data_normalizer(train_data, test_data)

Let's create the model using Keras. Keras is an API for Tensorflow. We have a few options for doing this, but we'll keep it simple for now. A model is built upon layers. We'll add two fully connected neural layers.

The number associated with the layer is the number of neurons in it. The first layer we'll use is a 'ReLU' (Rectified Linear Unit)' activation function. Since this is also the first layer, we need to specify input_size, which is the shape of an entry in our dataset.

After that, we'll finish with a softmax layer. Softmax is a type of logistic regression done for situations with multiple cases, like our 2 possible groups: 'Legendary' and 'Not Legendary'. With this we delineate the possible identities of the Pokémon into 2 probability groups corresponding to the possible labels:

In [34]:
length = train_data.shape[1]

model = keras.Sequential()
model.add(keras.layers.Dense(500, activation='relu', input_shape=[length,]))
model.add(keras.layers.Dense(2, activation='softmax'))

# Compile and Evaluate Model #3 
Once we have decided on the specifics of our model, we need to do two processes: Compile the model and fit the data to the model.

We can compile the model like so:

In [35]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Here we're just feeding three parameters to model.compile. We pick an optimizer, which determines how the model is updated as it gains information, a loss function, which measures how accurate the model is as it trains, and metrics, which specifies which information it provides so we can analyze the model.

The optimizer we're using is the Stochastic Gradient Descent (SGD) optimization algorithm, but there are others available. For our loss we're using sparse_categorical_crossentropy. If our values were one-hot encoded, we would want to use "categorial_crossentropy" instead.

Then we have the model fit our training data:

In [36]:
model.fit(train_data, train_labels, epochs=400)

Epoch 1/400
18/18 [==============================] - 2s 3ms/step - loss: 0.6196 - accuracy: 0.7895
Epoch 2/400
18/18 [==============================] - 0s 3ms/step - loss: 0.4763 - accuracy: 0.9246
Epoch 3/400
18/18 [==============================] - 0s 3ms/step - loss: 0.3985 - accuracy: 0.9263
Epoch 4/400
18/18 [==============================] - 0s 4ms/step - loss: 0.3532 - accuracy: 0.9263
Epoch 5/400
18/18 [==============================] - 0s 3ms/step - loss: 0.3244 - accuracy: 0.9263
Epoch 6/400
18/18 [==============================] - 0s 5ms/step - loss: 0.3052 - accuracy: 0.9263
Epoch 7/400
18/18 [==============================] - 0s 4ms/step - loss: 0.2916 - accuracy: 0.9263
Epoch 8/400
18/18 [==============================] - 0s 4ms/step - loss: 0.2814 - accuracy: 0.9263
Epoch 9/400
18/18 [==============================] - 0s 4ms/step - loss: 0.2733 - accuracy: 0.9263
Epoch 10/400
18/18 [==============================] - 0s 4ms/step - loss: 0.2664 - accuracy: 0.9263
Epoch 11/

18/18 [==============================] - 0s 3ms/step - loss: 0.0832 - accuracy: 0.9895
Epoch 84/400
18/18 [==============================] - 0s 5ms/step - loss: 0.0822 - accuracy: 0.9895
Epoch 85/400
18/18 [==============================] - 0s 5ms/step - loss: 0.0812 - accuracy: 0.9895
Epoch 86/400
18/18 [==============================] - 0s 5ms/step - loss: 0.0802 - accuracy: 0.9912
Epoch 87/400
18/18 [==============================] - 0s 8ms/step - loss: 0.0793 - accuracy: 0.9912
Epoch 88/400
18/18 [==============================] - 0s 8ms/step - loss: 0.0784 - accuracy: 0.9912
Epoch 89/400
18/18 [==============================] - 0s 5ms/step - loss: 0.0775 - accuracy: 0.9930
Epoch 90/400
18/18 [==============================] - 0s 6ms/step - loss: 0.0767 - accuracy: 0.9930
Epoch 91/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0758 - accuracy: 0.9930
Epoch 92/400
18/18 [==============================] - 0s 5ms/step - loss: 0.0750 - accuracy: 0.9930
Epoch 93/400


18/18 [==============================] - 0s 3ms/step - loss: 0.0320 - accuracy: 0.9947
Epoch 246/400
18/18 [==============================] - 0s 2ms/step - loss: 0.0318 - accuracy: 0.9965
Epoch 247/400
18/18 [==============================] - 0s 2ms/step - loss: 0.0317 - accuracy: 0.9947
Epoch 248/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0316 - accuracy: 0.9965
Epoch 249/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0315 - accuracy: 0.9965
Epoch 250/400
18/18 [==============================] - 0s 2ms/step - loss: 0.0314 - accuracy: 0.9965
Epoch 251/400
18/18 [==============================] - 0s 2ms/step - loss: 0.0313 - accuracy: 0.9947
Epoch 252/400
18/18 [==============================] - 0s 2ms/step - loss: 0.0312 - accuracy: 0.9965
Epoch 253/400
18/18 [==============================] - 0s 2ms/step - loss: 0.0311 - accuracy: 0.9947
Epoch 254/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0310 - accuracy: 0.9965
Epoc

The three parameters model.fit needs are our training data, our training labels, and the number of epochs. One epoch is when the model has iterated over every sample once. Essentially the number of epochs is equal to the number of times we want to cycle through the data. We'll start with just 1 epoch, and then show that increasing the epoch improves the results.

# Test it against our training data

In [38]:
loss_value, accuracy_value = model.evaluate(test_data, test_labels)
print(f'Our test accuracy was {accuracy_value}')

5/5 [==============================] - 0s 2ms/step - loss: 0.0333 - accuracy: 0.9801
Our test accuracy was 0.9801324605941772


model.evaluate will evaluate how strong our model is with the test data, and report that in the form of loss value and accuracy value (since we specified accuracy in our selected_metrics variable when we compiled the model). We'll just focus on our accuracy for now. With an accuracy of ~98%, it's not perfect, but it's very accurate.

We can also use our model to predict specific Pokémon, or at least have it tell us which status the Pokémon is most likely to have, with model.predict. All it needs to predict a Pokémon is the data for that Pokémon itself. We're providing that by selecting a certain index of test_data:

In [39]:
def predictor(test_data, test_labels, index):
    prediction = model.predict(test_data)
    if np.argmax(prediction[index]) == test_labels[index]:
        print(f'This was correctly predicted to be a \"{test_labels[index]}\"!')
    else:
        print(f'This was incorrectly predicted to be a \"{np.argmax(prediction[index])}\". It was actually a \"{test_labels[index]}\".')
        return(prediction)

Let's look at one of the more well-known legendary Pokémon: Mewtwo. He's number 150 in the list of Pokémon, so we'll look at index 149:

In [40]:
predictor(test_data, test_labels, 149)

This was correctly predicted to be a "1"!
